In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
import os
import japanize_matplotlib

In [ ]:
def get_sorted_csv_files_train(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_train_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def get_sorted_csv_files_test(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_test_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def read_csv_files(directory, file_list):
    data_frames = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    return data_frames

In [ ]:
network_name = "result_network10_4"
seed="seed0"
system_name = "BO_system_CT"
directory_path=os.path.join(network_name, system_name,"true_estimate",seed)
# directory_path=os.path.join(network_name, system_name,"true_estimate_test_",seed)

# sorted_file_list = get_sorted_csv_files_train(directory_path)
sorted_file_list = get_sorted_csv_files_test(directory_path)
data_frame_list = read_csv_files(directory_path, sorted_file_list)

# データフレームのリストを表示
for df in data_frame_list:
    print(df)

In [ ]:
def plot_true_estimate(data_frame_list,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    for idx, df in enumerate(data_frame_list):
        color = cmap(idx / len(data_frame_list))
        # plt.scatter(df['true'], df['estimate'], label=f'File {idx}', color=color)
        plt.scatter(df['observed'], df['estimated'], color=color,marker=".",alpha=0.5)
    
    # y=x の灰色の直線をプロット
    min_val = min(df['observed'].min() for df in data_frame_list)
    max_val = max(df['observed'].max() for df in data_frame_list)
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--')
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('True')
    plt.ylabel('Estimate')
    plt.title(title)
    # plt.legend()
    plt.show()

In [ ]:
plot_true_estimate(data_frame_list, title='True/Estimate CT test')

In [ ]:

for i,df in enumerate(data_frame_list):
    plt.scatter([i for j in range(df.shape[0])],df["observed"],marker=".")


In [ ]:
def plot_true_estimate_one(data_frame_list,index=0,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    # for idx, df in enumerate(data_frame_list):
    idx=index
    df=data_frame_list[idx]
    color = cmap(idx / len(data_frame_list))
    plt.scatter(df['observed'], df['estimated'], color=color,marker=".",alpha=1.0)
    
    # y=x の灰色の直線をプロット
    min_val = min(df['observed'].min() for df in data_frame_list)
    max_val = max(df['observed'].max() for df in data_frame_list)
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--')
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('True')
    plt.ylabel('Estimate')
    plt.title(title)
    # plt.legend()
    plt.show()

In [ ]:
plot_true_estimate_one(data_frame_list, 0, title='True vs Estimate CT(0)')
plot_true_estimate_one(data_frame_list, 99, title='True vs Estimate CT(99)')

In [ ]:
def plot_true_histgram(data_frame_list,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    for idx, df in enumerate(data_frame_list):
        color = cmap(idx / len(data_frame_list))
        # plt.scatter(df['true'], df['estimate'], label=f'File {idx}', color=color)
        plt.hist(df['observed'], bins=100,color=color,alpha=0.5,log=True)
    
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('Curie Temperature')
    plt.ylabel('Count')
    plt.title(title)
    # plt.legend()
    plt.show()

def plot_histogram_difference(data_frame_list, title='Histogram Difference'):
    if len(data_frame_list) < 2:
        raise ValueError("data_frame_list must contain at least two data frames.")
    
    # 最初と最後のデータフレームを取得
    df_first = data_frame_list[0]
    df_last = data_frame_list[-1]
    
    # ヒストグラムを計算
    hist_first, bins_first = np.histogram(df_first['observed'], bins=100)
    hist_last, bins_last = np.histogram(df_last['observed'], bins=bins_first)
    
    # 差分を計算
    hist_diff =  hist_first - hist_last
    print(hist_diff)
    print(sum(hist_diff))
    
    # プロット
    plt.figure(figsize=(7, 6))
    plt.hist(bins_first[:-1], bins_first, weights=hist_diff, alpha=0.5)
    
    plt.xlabel('True')
    plt.ylabel('Difference in Count')
    plt.title(title)
    plt.show()

In [ ]:
plot_true_histgram(data_frame_list, title='True histgram CT')

In [ ]:
plot_histogram_difference(data_frame_list, title='Histogram searched CT')

In [ ]:
def calculate_mae(data_frame_list):
    mae_list = []
    for df in data_frame_list:
        mae = np.mean(np.abs(df['observed'] - df['estimated']))
        mae_list.append(mae)
    return mae_list

def plot_mae(mae_list):
    plt.figure(figsize=(10, 6))
    plt.plot(mae_list, marker='o', linestyle='-', color='b')
    plt.xlabel('File Index')
    plt.ylabel('MAE')
    plt.title('Mean Absolute Error (MAE) for Each File')
    plt.show()

In [ ]:
# MAEを計算
mae_list = calculate_mae(data_frame_list)
# MAEをプロット
plot_mae(mae_list)